In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2

from tqdm.notebook import tqdm
from sklearn import metrics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

In [3]:
df=pd.read_csv("parkinson_disease.csv")
pd.set_option("display.max_columns",10)
df.sample(5)

,id,gender,PPE,DFA,RPDE,...,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
426,142,1,0.35512,0.69771,0.50633,...,41.3807,25.0060,19.7864,83.9067,1
190,63,1,0.79855,0.76093,0.69013,...,4.2521,6.0925,10.8435,60.8122,1
44,14,0,0.86478,0.76570,0.37052,...,3.6054,5.3050,4.6628,51.6426,0
189,63,1,0.82749,0.76107,0.64650,...,3.3428,7.2489,6.5402,28.3752,1
701,233,0,0.85464,0.69202,0.26806,...,11.5822,11.7430,10.1084,7.3975,1


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Columns: 755 entries, id to class
dtypes: float64(749), int64(6)
memory usage: 4.4 MB


In [26]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,756.0,125.500000,72.793721,0.000000,62.750000,125.500000,188.250000,251.00000
gender,756.0,0.515873,0.500079,0.000000,0.000000,1.000000,1.000000,1.00000
PPE,756.0,0.746284,0.169294,0.041551,0.762833,0.809655,0.834315,0.90766
DFA,756.0,0.700414,0.069718,0.543500,0.647053,0.700525,0.754985,0.85264
RPDE,756.0,0.489058,0.137442,0.154300,0.386537,0.484355,0.586515,0.87123
...,...,...,...,...,...,...,...,...
tqwt_kurtosisValue_dec_33,756.0,12.375335,16.341665,1.628700,3.114375,4.741450,12.201325,73.53220
tqwt_kurtosisValue_dec_34,756.0,14.799230,15.722502,1.861700,3.665925,6.725700,21.922050,62.00730
tqwt_kurtosisValue_dec_35,756.0,14.751559,14.432979,1.955900,3.741275,7.334250,22.495175,57.54430
tqwt_kurtosisValue_dec_36,756.0,31.481110,34.230991,2.364000,3.948750,10.637250,61.125325,156.42370


In [28]:
df.isnull().sum().sum()

np.int64(0)

In [30]:
df = df.groupby("id").mean().reset_index()
#df_grouped = df.groupby('id').mean().reset_index()

df.drop("id",axis=1,inplace=True)


In [ ]:
columns=list(df.columns)
for col in columns:
    if col=="class":
        continue
    filtered_columns=[col]
    for col1 in df.columns:
        if((col==col1) | (col=="class")):
            continue
        val=df[col].corr(df[col1])
        if val>0.7:
            columns.remove(col1)
            continue
        else:
            filtered_columns.append(col1)
    df=df[filtered_columns]
df.shape

In [17]:
X=df.drop("class",axis=1)
X_norm=MinMaxScaler().fit_transform(X)
selector=SelectKBest(chi2,k=30)
selector.fit(X_norm,df["class"])
filtered_columns=selector.get_support()
filtered_data=X.loc[:,filtered_columns]
filtered_data["class"]=df["class"]
df=filtered_data
df.shape

(252, 31)